In [2]:
import os 
from sys import argv
import random as rd
from copy import deepcopy

def load_motifsEF(filename):
    file=open(filename)
    seq=file.read().split("MOTIF")
    seq=seq[1:]
    motifs={}
    infos={}
    for s in range(len(seq)):
        t=seq[s].strip().split("\n")
        motifs[t[0]]=t[2:]  #remove HOCOMOCO URL
        infos[t[0].split('_')[0]]=t[:2]
    for m in motifs:
        tdict={'A':[],'C':[],'G':[],'T':[],'E':[],'F':[]}
        for pos in range(len(motifs[m])):
            tmp=motifs[m][pos].strip().split("\t")
            tdict['A']+=[float(tmp[0])]
            tdict['C']+=[float(tmp[1])]
            tdict['G']+=[float(tmp[2])]
            tdict['T']+=[float(tmp[3])]
            tdict['E']+=[float(tmp[4])]
            tdict['F']+=[float(tmp[5])]
        motifs[m]=tdict
    
    return motifs,infos


def load_motifs(filename):
    file=open(filename)
    seq=file.read().split("MOTIF")
    seq=seq[1:]
    motifs={}
    infos={}
    for s in range(len(seq)):
        t=seq[s].strip().split("\n")
        motifs[t[0]]=t[2:-1]  #remove HOCOMOCO URL
        infos[t[0].split('_')[0]]=t[:2]
    for m in motifs:
        tdict={'A':[],'C':[],'G':[],'T':[]}
        for pos in range(len(motifs[m])):
            tmp=motifs[m][pos].strip().split("\t")
            tdict['A']+=[float(tmp[0])]
            tdict['C']+=[float(tmp[1])]
            tdict['G']+=[float(tmp[2])]
            tdict['T']+=[float(tmp[3])]
            #tdict['E']+=[float(tmp[4])]
            #tdict['F']+=[float(tmp[5])]
        motifs[m]=tdict
    
    return motifs,infos
def insertBaseToPWM(pwm):
    newpwm = deepcopy(pwm)
    k = len(pwm['A'])
    base = rd.choice(['E','E'])
    loc = rd.choice(range(k))
    freq = rd.choice(range(5,10))*0.1
    print base,loc,freq
    newpwm['E'] = [0]*k
    newpwm['F'] = [0]*k

    for c in newpwm.keys():
        newpwm[c][loc] = (1-freq)/(len(newpwm.keys())-1)
    newpwm[base][loc] = freq
    return newpwm

def randomkmer_typeEF(PWM):
    #take a PWM and randomly create a Kmer based on it
    # chance goes A->C->G->T->E
    kmer=""
    for pos in range(len(PWM['A'])):
        die=rd.randint(0,100000)
        if die<=PWM['A'][pos]*100000:
            kmer+='A'
        elif die<=(PWM['A'][pos]+PWM['C'][pos])*100000:
            kmer+='C'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos])*100000:
            kmer+='G'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos]+PWM['T'][pos])*100000:
            kmer+='T'
        elif die<=(PWM['A'][pos]+PWM['C'][pos]+PWM['G'][pos]+PWM['T'][pos]+PWM['E'][pos])*100000:
            kmer+='E'
        else:
            kmer+='F'
    return kmer

def loadFasta(filename):
    seqs = {}
    lines = open(filename).read().split('>')[1:]
    for line in lines:
        tmp = line.split('\n')
        seqs[tmp[0]] = ''.join(tmp[1:])
    return seqs

def insertoseqs(pwm,seqs,numseqtopick):
    #numseqtopick=10
    seqspicked = rd.sample(seqs.keys(),numseqtopick)
    
    
    #print seqspicked
    for s in seqspicked:
        kmer = randomkmer_typeEF(pwm)
        #print kmer
        string = list(seqs[s])
        insertloc = rd.randrange(0,len(string)-len(kmer))
        #print insertloc
        for i in xrange(len(kmer)):
            string[insertloc + i]=kmer[i]
    
        seqs[s] = ''.join(string)
    return seqs
def printPWM(pwm,name,outfile):
    target = open(outfile,'w')
    target.write("MOTIF\t"+name+'\n')
    for loc in range(len(pwm['A'])):
        line = []
        for base in ['A','C','G','T','E','F']:
            line += [str(pwm[base][loc])]
        line = '\t'.join(line)
        target.write(line+'\n')
    target.close()
    return
    

In [7]:
def main():
    prefix = "./test_data_typeEF/"
    fastafile = prefix + "CEBPB.test.typeEF.sequence.faa"
    motifs,infos = load_motifs("HOCOMOCOv10_HUMAN_mono_meme_format.meme")
    mtochoose = rd.choice(motifs.keys())
    print mtochoose
    pwm = motifs[mtochoose]
    #print mtochoose,pwm
    newpwm = insertBaseToPWM(pwm)
    printPWM(newpwm,mtochoose+"_inserted",prefix+mtochoose+'_inserted.typeEF.meme')
    #print newpwm
    
    seqs = loadFasta(fastafile)
    testingabundances = [0.005,0.01,0.02,0.05]
    #len(seqs)
    for rate in testingabundances:
        outputseqfile = fastafile.replace('faa',str(rate)+'.faa')
        print outputseqfile
        target = open(outputseqfile,'w')
        numseqtopick = int(len(seqs)*rate)
        print numseqtopick
        #seqstoinsert = rd.sample(seqs.keys())
        seqs = insertoseqs(newpwm,seqs,numseqtopick)
        for s in seqs:
            target.write(">"+s+'\n')
            target.write(seqs[s]+'\n')
        target.close()
        
    
    #now run mEpigram on this set
    #ada
    
    #look at the results to see whether the inserted motif is in it
    # check how many false positives
        
    return
main()

FOXL1_HUMAN.H10MO.D
E 17 0.5
./test_data_typeEF/CEBPB.test.typeEF.sequence.0.005.faa
5
./test_data_typeEF/CEBPB.test.typeEF.sequence.0.01.faa
10
./test_data_typeEF/CEBPB.test.typeEF.sequence.0.02.faa
20
./test_data_typeEF/CEBPB.test.typeEF.sequence.0.05.faa
50


In [59]:
import os
    
    
    

In [47]:
insertoseq(newpwm,seqs,10)

['chr8_56843218', 'chr7_141111087', 'chr9_1871891', 'chr22_29517169', 'chr6_158028818', 'chr19_45245180', 'chr5_96566816', 'chr2_120124291', 'chr3_16592408', 'chr16_77618159']
AFCGGAAAA
GEAGGAAGT
TECGGAAGC
AEAGGATGA
AGCGGAAAT
TTCGGAAGT
GEGGGAGCT
AEAGGATGA
GEAGGATAA
AEAGGAAGT


In [48]:
seqs['chr8_56843218']

'GTGTGAAATTAFCGGAAAAAGAGTTATGCAAATGGGGATGCGGCACGTGGTGGAGGCTTCTTGCAG'

In [61]:
os.system("python mepigram_wrapper3.py")

256